**Next_Word_Prediction**

# Import libraries

In [1]:
!pip install tensorflow-gpu
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
  error: subprocess-exited-with-error
  
  × Running setup.py install for tensorflow-gpu did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for tensorflow-gpu ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> tensorflow-gpu

note: This is an issue with the package mentioned above, not pip.
hint: See above for outpu

In [2]:
from google.colab import files
uploaded=files.upload()

Saving text_dataset.txt to text_dataset.txt


# Load and PreProcess the data

In [3]:
file=open("text_dataset.txt","r",encoding="utf8")
#store file in list
line=[]
for i in file:
  line.append(i)
# Convert list to string
data = ""
for i in line:
  data = ' '. join(line)
#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  
#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:500]

'The Project Gutenberg eBook of A Room With A View, by E. M. Forster This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using t'

In [4]:
len(data)

387734

# Apply tokenization

In [5]:
token = Tokenizer()
token.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(token, open('token.pkl', 'wb'))

sequence_data = token.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 114, 105, 573, 4, 5, 115, 19, 5, 195, 46, 310, 1223, 1839, 33]

In [6]:
len(sequence_data)

70167

In [7]:
vocab_size = len(token.word_index) + 1
vocab_size

7866

In [8]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:15]

The Length of sequences are:  70164


array([[   1,  114,  105,  573],
       [ 114,  105,  573,    4],
       [ 105,  573,    4,    5],
       [ 573,    4,    5,  115],
       [   4,    5,  115,   19],
       [   5,  115,   19,    5],
       [ 115,   19,    5,  195],
       [  19,    5,  195,   46],
       [   5,  195,   46,  310],
       [ 195,   46,  310, 1223],
       [  46,  310, 1223, 1839],
       [ 310, 1223, 1839,   33],
       [1223, 1839,   33,  573],
       [1839,   33,  573,   17],
       [  33,  573,   17,   20]])

In [9]:
x = []
y = []

for i in sequences:
    x.append(i[0:3])
    y.append(i[3])
    
X = np.array(x)
y = np.array(y)



In [10]:
print("Data: ", x[:10])
print("Response: ", y[:10])

Data:  [array([  1, 114, 105]), array([114, 105, 573]), array([105, 573,   4]), array([573,   4,   5]), array([  4,   5, 115]), array([  5, 115,  19]), array([115,  19,   5]), array([ 19,   5, 195]), array([  5, 195,  46]), array([195,  46, 310])]
Response:  [ 573    4    5  115   19    5  195   46  310 1223]


In [11]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# Creating the model

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             78660     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 7866)              7873866   
                                                                 
Total params: 21,001,526
Trainable params: 21,001,526
Non-trainable params: 0
_________________________________________________________________


# Train The Model

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])
model.fit(X, y, epochs=150, batch_size=70, callbacks=[checkpoint])

Epoch 1/150
1003/1003 [==============================] - ETA: 0s - loss: 6.7470 - accuracy: 0.0471
Epoch 1: loss improved from inf to 6.74704, saving model to next_words.h5
1003/1003 [==============================] - 40s 29ms/step - loss: 6.7470 - accuracy: 0.0471
Epoch 2/150
1003/1003 [==============================] - ETA: 0s - loss: 6.2108 - accuracy: 0.0718
Epoch 2: loss improved from 6.74704 to 6.21081, saving model to next_words.h5
1003/1003 [==============================] - 21s 21ms/step - loss: 6.2108 - accuracy: 0.0718
Epoch 3/150
1002/1003 [============================>.] - ETA: 0s - loss: 5.8002 - accuracy: 0.0955
Epoch 3: loss improved from 6.21081 to 5.80033, saving model to next_words.h5
1003/1003 [==============================] - 21s 21ms/step - loss: 5.8003 - accuracy: 0.0955
Epoch 4/150
1002/1003 [============================>.] - ETA: 0s - loss: 5.4985 - accuracy: 0.1096
Epoch 4: loss improved from 5.80033 to 5.49856, saving model to next_words.h5
1003/1003 [======

# Prediction

In [15]:
from tensorflow.keras.models import load_model
# Load the model and tokenizer
model = load_model('next_words.h5')
token = pickle.load(open('token.pkl', 'rb'))
def Predict_NextWords(model, token, text):

  sequence = token.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in token.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [16]:
while(True):
  input_text = input("Enter your input line: ")
  
  if input_text == "0":
      print("Execution completed")
      break
  
  else:
      try:
          input_text = input_text.split(" ")
          input_text = input_text[-3:]
          print(input_text)
        
          Predict_NextWords(model, token, input_text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue

Enter your input line: A Room With A
['Room', 'With', 'A']
1/1 [==============================] - 1s 951ms/step
view
Enter your input line: The father did not see it; the son acknowledged it, not by another
['not', 'by', 'another']
1/1 [==============================] - 0s 30ms/step
bow
Enter your input line: Beyond them stood the
['them', 'stood', 'the']
1/1 [==============================] - 0s 30ms/step
unreliable
Enter your input line: to our depressing
['to', 'our', 'depressing']
1/1 [==============================] - 0s 30ms/step
signora
Enter your input line: that he is a
['he', 'is', 'a']
1/1 [==============================] - 0s 24ms/step
socialist
Enter your input line: And presumably he has
['presumably', 'he', 'has']
1/1 [==============================] - 0s 26ms/step
brought
Enter your input line: The little old
['The', 'little', 'old']
1/1 [==============================] - 0s 34ms/step
lady
Enter your input line: 0
Execution completed
